## Compare classic CNN with quantum-classic CNN on CIRAF-10 dataset

In [1]:
import torch
import numpy as np

use_cuda = torch.cuda.is_available()
print(f"CUDA available: {use_cuda}")

/home/jacek/ingenii/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: False


In [3]:
# Load dataset

import random
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

random.seed(42)

batch_size = 16
valid_size = 0.2

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_data = datasets.CIFAR10('data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False, download=True, transform=transform)


num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# Create classic model architecture

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from ingenii_quantum_fork.hybrid_networks.layers import QuantumLayer2D

class CNN(nn.Module):

    CHANNELS = [32, 64, 128, 128, 256, 256]

    def __init__(self, quantum=False):
        super(CNN, self).__init__()
        self.quantum = quantum

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=self.CHANNELS[0], kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=self.CHANNELS[0], out_channels=self.CHANNELS[1], kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=self.CHANNELS[1], out_channels=self.CHANNELS[2], kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=self.CHANNELS[2], out_channels=self.CHANNELS[3], kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(in_channels=self.CHANNELS[3], out_channels=self.CHANNELS[4], kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(in_channels=self.CHANNELS[4], out_channels=self.CHANNELS[5], kernel_size=3, padding=1)

        self.quantum_layer = QuantumLayer2D(
            shape=(4, 4), num_filters=2, gates_name='G3', num_gates=300, num_features=32, tol=1e-6, stride=1, shots=409)

        self.bn1 = nn.BatchNorm2d(self.CHANNELS[0])
        self.bn3 = nn.BatchNorm2d(self.CHANNELS[2])
        self.bn5 = nn.BatchNorm2d(self.CHANNELS[4])

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(4096, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)

        self.drop = nn.Dropout(p=0.5)

    def forward(self, x):
        if self.quantum and x.dim() == 1:
            x = x.unsqueeze(-1)

        x = F.relu(self.bn1(self.conv1(x)))
        if self.quantum:
            quantum_conv = self.quantum_layer(x)
            conv = F.relu(self.conv2(x))
            x = self.pool(conv + quantum_conv)
        else:
            x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))
        x = self.pool(F.relu(self.conv6(x)))

        x = x.view(x.size(0), -1)

        x = self.drop(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.drop(x)
        x = self.fc3(x)

        return x


model = CNN(quantum=False)
if use_cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=.001)

In [ ]:
# Train and test model

n_epochs = 30
valid_loss_min = np.Inf

for epoch in range(1, n_epochs + 1):

    train_loss = 0.0
    valid_loss = 0.0
    valid_acc = 0.0
    
    model.train()
    for data, target in train_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    total = 0
    model.eval()
    for data, target in valid_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        loss = criterion(output, target)
        valid_loss += loss.item()*data.size(0)
        _, predicted = torch.max(output.data, 1)
        total += len(target)
        valid_acc += (predicted == target).sum().item()
    
    train_loss = train_loss / len(train_loader.dataset)
    valid_loss = valid_loss / len(valid_loader.dataset)
    valid_acc = valid_acc / total

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tValidation Acc: {:.6f}'.format(
        epoch, train_loss, valid_loss, valid_acc))

total = 0
test_acc = 0.0
model.eval()
for data, target in valid_loader:
    if use_cuda:
        data, target = data.cuda(), target.cuda()
    output = model(data)
    loss = criterion(output, target)
    valid_loss += loss.item()*data.size(0)
    _, predicted = torch.max(output.data, 1)
    total += len(target)
    test_acc += (predicted == target).sum().item()
test_acc = test_acc / total
print("Test accuracy: {:.6f}".format(test_acc))

